In [17]:
path = '/home/wjunneng/Python/ForecastScore/'

submission_s1 = '/data/test_s1/submission_s1.csv'
submission_s1_sample_mean_median = '/data/test_s1/submission_s1_sample_mean_median.csv'
submission_s1_sample_svm = '/data/test_s1/submission_s1_sample_svm.csv'
submission_s1_sample_xgb = '/data/test_s1/submission_s1_sample_xgb.csv'
submission_s1_sample_lgb = '/data/test_s1/submission_s1_sample_lgb.csv'

submission_s1_sample_mean_median1 = '/data/test_s1/submission_s1_sample_mean_median1.csv'

course = '/data/train_s1/course.csv'
student = '/data/train_s1/student.csv'
exam_score = '/data/train_s1/exam_score.csv' 
all_knowledge = '/data/train_s1/all_knowledge.csv'
course1_exams = '/data/train_s1/course1_exams.csv'
course2_exams = '/data/train_s1/course2_exams.csv'
course3_exams = '/data/train_s1/course3_exams.csv'
course4_exams = '/data/train_s1/course3_exams.csv'
course5_exams = '/data/train_s1/course5_exams.csv'
course6_exams = '/data/train_s1/course6_exams.csv'
course7_exams = '/data/train_s1/course7_exams.csv'
course8_exams = '/data/train_s1/course8_exams.csv'

submission_s1_sample = '/data/submission_s1_sample.csv'

In [18]:
import numpy as np
import pandas as pd


def get_train_s1(file_name, tag):
    """
    返回train_s1文件夹下的文件
    :param file_name: 文件名称
    :param tag: 返回的数据类型
    :return:
    """
    if tag == 'np':
        return np.loadtxt(path + file_name, delimiter=',', dtype=np.str)
    elif tag == 'pd':
        return pd.read_csv(path + file_name)
    else:
        return "请检查文件名/需要返回的数据类型"

def get_test_s1(file_name, tag):
    """
    返回test_s1文件夹下的文件
    :param file_name: 文件名称
    :param tag: 返回的数据类型
    :return:
    """
    if tag == 'np':
        return np.loadtxt(path + file_name, delimiter=',', dtype=np.str)
    elif tag == 'pd':
        return pd.read_csv(path + file_name)
    else:
        return "请检查文件名/需要返回的数据类型"

def get_sample(file_name, tag):
    """
    返回范例文件夹下的文件
    :param file_name: 文件名称
    :param tag: 返回的数据类型
    :return:
    """
    if tag == 'np':
        return np.loadtxt(path + file_name, delimiter=',', dtype=np.str)
    elif tag == 'pd':
        return pd.read_csv(path + file_name)
    else:
        return "请检查文件名/需要返回的数据类型"


In [31]:
exam_score_data = get_train_s1(exam_score, 'pd')

reuslt = []
for i in ['course1', 'course2', 'course3', 'course4', 'course5', 'course6', 'course7', 'course8']:
    exam_score_course = exam_score_data[exam_score_data.course == i]
    for exam_score_course_student in exam_score_course.groupby('student_id'):
        print('--'+ str(exam_score_course_student[1].shape) +'--')
        tmp = exam_score_course_student[1].score
        if tmp.values.min() <= 70 and tmp.values.min() != 0:
            print(exam_score_course_student[1])

--(18, 4)--
--(18, 4)--
      student_id   course   exam_id  score
384       101790  course1  pNdxJnEf     77
884       101790  course1  5Sh31xmN     75
1384      101790  course1  4CHUDKzQ     72
1884      101790  course1  as7yltQJ     76
2384      101790  course1  AFnBGQxE     77
2884      101790  course1  8wUj4LGF     82
3384      101790  course1  wUfEa8v2     70
3884      101790  course1  hFZCYHcM     75
4384      101790  course1  03FybMPz     73
4884      101790  course1  uSax3LOe     89
5384      101790  course1  ZOyuld3U     77
5884      101790  course1  Bcn8WGsT     75
6384      101790  course1  wjcFO1Nq     79
6884      101790  course1  Rdovw3C7     79
7384      101790  course1  giCyVrdq     82
7884      101790  course1  IVEd5SkM     74
8384      101790  course1  YFZJ4rei     80
8884      101790  course1  EZtGMeKm     76
--(18, 4)--
--(18, 4)--
      student_id   course   exam_id  score
243       107330  course1  pNdxJnEf     73
743       107330  course1  5Sh31xmN     69
1243  